# Generates a series of PNGs to be stitched together with ffmpeg or similar

In [1]:
import matplotlib.pyplot as plt
import netCDF4
import numpy as np
import pandas as pd
from mpl_toolkits.basemap import Basemap
from tqdm import tqdm
###################################################
import torch
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
###################################################
import torch
import torchvision as tv
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
###################################################
from livelossplot import PlotLosses
import hiddenlayer as hl
###################################################
import pickle
import os
###################################################
import scipy as sp
import scipy.fftpack

In [2]:
temp_nc = netCDF4.Dataset('../eniko/data/tas/tas_day_MPI-ESM-MR_rcp85_r1i1p1_g025.nc')
temp_ncdata_3D = np.array(temp_nc.variables['tas'])
temp_ncdata_2D = temp_ncdata_3D.reshape(temp_ncdata_3D.shape[0], temp_ncdata_3D.shape[1]*temp_ncdata_3D.shape[2])        

dates = pd.to_datetime(netCDF4.num2date(temp_nc.variables['time'][:], temp_nc.variables['time'].units)).year
temp_ncdata_df = pd.DataFrame(temp_ncdata_2D, index = dates)

In [3]:
train_start = 2025
train_end = 2100

test_start = 1975
test_end = 1978

max_fts = 10
batch_size = 100

In [4]:
# PCA type
train_idx = temp_ncdata_df.index.isin(range(train_start,train_end+1))
train_samples = temp_ncdata_df.iloc[train_idx]
train_years = temp_ncdata_df.index[temp_ncdata_df.index.isin(range(train_start,train_end+1))]

test_idx = temp_ncdata_df.index.isin(range(test_start,test_end+1))
test_samples = temp_ncdata_df.iloc[test_idx]
test_years = temp_ncdata_df.index[temp_ncdata_df.index.isin(range(test_start,test_end+1))]

# Standardise the training samples
train_samples = torch.tensor(train_samples.values)#.double()
test_samples = torch.tensor(test_samples.values)#.double()

mu, sig = train_samples.mean(), train_samples.std()

# train_samples = train_samples.sub(mu).div(sig)
# test_samples = test_samples.sub(mu).div(sig)

mmin = min(train_samples.min(),test_samples.min())
mmax = max(train_samples.max(),test_samples.max())

# # Normalise instead
train_samples = (train_samples - mmin) / (mmax - mmin)
test_samples = (test_samples - mmin) / (mmax - mmin)



train_data = DataLoader(train_samples, batch_size=batch_size)
test_data = DataLoader(test_samples, batch_size=batch_size)


# Load autoencoder model

In [7]:
class AutoEncoder(nn.Module):
    def __init__(self, layers):
        super(AutoEncoder,self).__init__()
        assert len(layers) == 5, 'Not the right number of layer numbers'
        self.encoder = nn.Sequential(
            nn.Conv2d(1, layers[0], kernel_size=(3,5))
            ,nn.MaxPool2d(2)
            ,nn.ReLU()
            
            ,nn.Conv2d(layers[0], layers[1], kernel_size=(2,5))
            ,nn.MaxPool2d((2,3))
            ,nn.ReLU()
            
            ,nn.Conv2d(layers[1], layers[2], kernel_size=(2,3))
            ,nn.MaxPool2d(2)
            ,nn.ReLU()
            
            ,nn.Conv2d(layers[2], layers[3], kernel_size=(3,3))
            ,nn.MaxPool2d(2)
            ,nn.ReLU()
            
            ,nn.Conv2d(layers[3], layers[4], kernel_size=(2,3))
            ,nn.MaxPool2d(2)
        )
        
        self.decoder = nn.Sequential(
            
            nn.ConvTranspose2d(layers[4], layers[3], kernel_size=(3,4), stride=(3,4))
            ,nn.ReLU()
                    
            ,nn.ConvTranspose2d(layers[3], layers[2], kernel_size=(3,3), stride=(3,3))
            ,nn.ReLU()
            
            ,nn.ConvTranspose2d(layers[2], layers[1], kernel_size=(2,3), stride=(2,3))
            ,nn.ReLU()
            
#             ,nn.PixelShuffle(2)
            
            ,nn.ConvTranspose2d(layers[1], layers[0], kernel_size=2, stride=2)
            ,nn.ReLU()
                       
            ,nn.ConvTranspose2d(layers[0], 1, kernel_size=(2,2), stride=(2,2))
            ,nn.Sigmoid()
        )
        
    def forward(self, x):
        # Encoder part
        y = self.encoder(x)
        # Decoder part
        x = self.decoder(y)
        return y, x # x is the output of the autoencoder, y is the output of the encoder

In [8]:
bottleneck=256

In [15]:
# This will train the model
num_epochs = 20
bottleneck= 256

# Parameters for regularisation - what should these be??
tv_lambda = 0.01
# l1_lambda = 1.0

reg_TV = True
reg_BN = False
# ae_model = 'data/models/autoencoder/ae_reg_TV_{}_BN_{}_{}_{}_to_{}_{}_epochs_tv_lambda_{}.pickle'.format(\
#                                                                                                 str(reg_TV)
#                                                                                                 ,str(reg_BN)
#                                                                                                 ,bottleneck
#                                                                                                 ,train_start
#                                                                                                 ,train_end
#                                                                                                 ,num_epochs
#                                                                                                 ,tv_lambda)
ae_model = 'data/models/autoencoder/ae_reg_TV_True_BN_False_256_2025_to_2100_20_epochs_tv_lambda_0.001.pickle'
model = AutoEncoder(layers=[8,32,64,128,256])
model.load_state_dict(torch.load(ae_model))
model.eval()

AutoEncoder(
  (encoder): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
    (3): Conv2d(8, 32, kernel_size=(2, 5), stride=(1, 1))
    (4): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=0, dilation=1, ceil_mode=False)
    (5): ReLU()
    (6): Conv2d(32, 64, kernel_size=(2, 3), stride=(1, 1))
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): ReLU()
    (9): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): ReLU()
    (12): Conv2d(128, 256, kernel_size=(2, 3), stride=(1, 1))
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(256, 128, kernel_size=(3, 4), stride=(3, 4))
    (1): ReLU()
    (2): ConvTranspose2d(128, 64, kernel_size=(3, 3), 

In [6]:
with open('data/models/pca/pca_5_2025_to_2100.pickle', 'rb') as file:
    pca_model = pickle.load(file)

In [ ]:
long = [0, 360]
lat = [-90, 90]

import matplotlib.animation as animation

fig = plt.figure(figsize=(16,6))

my_map = Basemap(projection = 'cyl', llcrnrlat = lat[0], llcrnrlon = long[0], urcrnrlat = lat[1], urcrnrlon = long[1])
my_map.drawcoastlines()
my_map.drawparallels( np.arange(-90,90.01,30.0), labels = [1,0,0,0], fontsize = 12, linewidth = 0)
my_map.drawmeridians( np.arange(0.,360.,30.), labels = [0,0,0,1], fontsize = 12, linewidth = 0)

mynorm = plt.Normalize(vmin=0.0, vmax=1.0)

ims = []

for k in tqdm(range(test_samples.shape[0])[0:(365*3)]):

    # y, x = model(annual_samples[k].reshape(1,1,72,144))

    # h = my_map.imshow(x.detach().numpy().reshape(72,144), cmap = 'jet', norm=mynorm, interpolation = 'none', animated = True)


    transformed_img = pca_model.transform(test_samples[k].reshape(1,72*144).detach().numpy())
    inv = pca_model.inverse_transform(transformed_img)

    h = my_map.imshow(inv.reshape(72,144), cmap = 'jet', norm=mynorm, interpolation = 'none', animated = True)

    my_map.colorbar(h, size = "2%")#, pad="40%", ticks = range(cmin, cmax + 1, cint))

    plt.title("Near-surface air temperature")

    ims.append([h])



100%|██████████| 1095/1095 [00:42<00:00, 25.60it/s]


In [11]:
ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True, repeat_delay=1000)

ani.save('data/videos/pca/test_256.mp4')